# Welcome to the Geonotebook
GeoNotebook is an application that provides client/server enviroment with inteactive visualization and analysis capabilities using Jupyter notebook, GeoJS and other open source tools.

The example notesbooks in this directory will walk you through several of the features that the ```geonotebook``` plugin to Jupyter makes available. The first thing to know about is...

### The geonotebook object

The ```M``` object is inserted into the kernel automatically once the notebook is started.  This object lives inside the Python kernel's namespace and communicates information to (and receives information from) the GeoJS map. Note that nothing has been imported,  but the ```M``` variable is still available.

**Note:** If you are viewing a static version of this notebook you will NOT see the GeoJS map that is dynamically added to a running notebook.  Please see this [Screen shot](https://raw.githubusercontent.com/OpenGeoscience/geonotebook/master/screenshots/geonotebook.png) to get a sense of the running interface.

In [ ]:
M

### Set the map's center

The M object exposes a number of different functions for interacting with the map (which should be located to the right of standard jupyter python cells).

Executing the following cell should set the center of the map to New York State. 


In [1]:
# set_center's arguments are longitude, latitude, and zoom level
M.set_center(-74, 43, 6)

### What just happened?

It is important to understand that ```M.set_center(...)``` is a Python statement being made inside the Python kernel. It is using a remote procedure call to change the javascript map's location.

The standard Jupyter notebook has three components, (1) the client that makes up the notebook cells, (2) a web server that lists notebook files,  directories and serves notebook assets like HTML and CSS (3) a kernel that executes commands in the chosen language (in our case Python).

![what just happened](https://docs.google.com/drawings/d/e/2PACX-1vT60rVDypw7I5pGnfF0nO8vFh-bd-bFbTT1_mMfYrvV66xtdelgxKSogCWkluM0ca6Z62ZA8MzDAmFo/pub?w=1440&h=1080 "Comm Channel")

When you executed the previous cell the string "M.set_center(-74, 43, 6)" was transmitted over a web socket to the webserver,  then proxied through ZeroMQ to the IPykernel where it was evaluated as a Python expression. This is the standard way in which Jupyter notebook takes code from a web browser,  and executes it in an interactive shell (kernel). M is an object in the kernel,  and it has a function *set_center*.  That function executed and returned a [promise](https://pypi.python.org/pypi/promise),  which is why you see something in the cell output like ```<promise.promise.Promise at 0x7f567dd8f290>```

While the ```set_center``` function returns a promise,  it also has a side effect. This side effect uses a custom jupyter communication channel (or 'Comm') to tell the map to change its view port so the center is at (in this case) -74.0 latitude,  43.0 longitude,  with a zoom level of 6. 



## Widget example

One question you may immediately ask yourself is,  why not have the notebook cell talk to the map directly?  Why get python involved at all? Well, because ```M.set_center``` is just a Python function,  it can do things like leverage the existing widget extension to the notebook.  

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def map_widgets(lat=0.0, lon=0.0, zoom=4):
  M.set_center(lon, lat, zoom)
  
interact(map_widgets, lat=(-90.0, 90.0), lon=(-180.0, 180.0), zoom=(1, 9))

# Annotations

In addition to supporting Python to Map communications,  Geonotebook allows objects and events on the map to communicate back to the Python kernel. One of the primary ways in which this is used is through geonotebook annotations. 

On the toolbar,  next to the "CellToolbar" button,  there should be three additional buttons with a circle,  square and a polygon. Hovering over each of these reveals that they can be used to start a point, rectangle or polygon annotation. 

### Point annotations
Try clicking on the circle icon. Notice that the cursor,  when it hovers over the map,  is now a cross rather than an arrow. Click on the map and a circle annotation should appear. 

### Rectangle Annotations
Try clicking on the square icon. If you click on the map and hold down the left mouse button, then drag the mouse and release the left mouse button you should be able to create a rectangular annotation. 

### Polygon annotations
Try clicking on the polygon icon.  Single click on a series of points to begin creating a polygon annotation. Double click on a point and the final segment will be added completing the annotation.


Annotations inherit from [shapely](http://toblerity.org/shapely/manual.html) geometries, this means they support a wide range of spatial functions.

In [ ]:
p = M.layers.annotation.polygons[0]
p

You can get a list of coordinates for the polygon expressed in latitude and longitude

In [ ]:
# List the exterior coordinates of the annotation
# Expressed in latitude and longitude point pairs
list(p.exterior.coords)

Other properties like 'centroid' and 'area' are also available,  keeping in mind that all coordinates are measured in latitude/longitude. This means properties like 'area' will not have much meaning.  You can look at Shapely's [transform](http://toblerity.org/shapely/manual.html#shapely.ops.transform) method for information on how to translate these into to something more useful

In [ ]:
list(p.centroid.coords)

Here is an example of using shapely's transform method to convert coordinates from latitude/longitude (EPSG:4326)  to Albers equal area (AEA).  The resulting object gives area in units of meters squared 

In [ ]:
import pyproj
import shapely.ops as ops
from functools import partial

project = partial(pyproj.transform,
                  pyproj.Proj(init='EPSG:4326'), 
                  pyproj.Proj(proj='aea',
                    lat1=p.bounds[1],
                    lat2=p.bounds[3]))

ops.transform(project, p).area

In [ ]:
M.layers.annotation.clear_annotations()

## National Land Cover Dataset Example

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib.pylab import plt
import numpy as np
import pandas as pd

In [ ]:
legend = pd.DataFrame([
    (11, "Open Water", "#476BA0"),
    (12, "Perennial Ice/Snow", "#D1DDF9"),
    (21, "Developed, Open Space","#DDC9C9"),
    (22, "Developed, Low Intensity", "#D89382"),
    (23, "Developed, Medium Intensity", "#ED0000"),
    (24, "Developed High Intensity", "#AA0000"),
    (31, "Barren Land (Rock/Sand/Clay)", "#B2ADA3"),
    (41, "Deciduous Forest", "#68AA63"),
    (42, "Evergreen Forest", "#1C6330"),
    (43, "Mixed Forest", "#B5C98E"),
    (51, "Dwarf Scrub", "#A58C30"),
    (52, "Shrub/Scrub", "#CCBA7C"),
    (71, "Grassland/Herbaceous", "#E2E2C1"),
    (72, "Sedge/Herbaceous", "#C9C977"),
    (73, "Lichens", "#99C147"),
    (74, "Moss", "#77AD93"),
    (81, "Pasture/Hay", "#DBD83D"),
    (82, "Cultivated Crops", "#AA7028"),
    (90, "Woody Wetlands", "#BAD8EA"),
    (95, "Emergent Herbaceous Wetlands","#70A3BA")],
    columns=["Code", "Desc", "Color"])

def highlight(e):
    return 'background-color: {}'.format(e)


In [ ]:
from geonotebook.wrappers import RasterData

rd = RasterData("/data/kotfic/nlcd_2011_landcover_2011_edition_2014_10_10.tif")

colormap = legend[["Code", "Color"]].rename(columns={
    "Code": "quantity", "Color": "color"}).to_dict("records")

In [ ]:
M.add_layer(rd[1], colormap=colormap, opacity=0.7)

### What just happened here?

![](https://docs.google.com/drawings/d/e/2PACX-1vSFysHt4BmP1etUUJFtPGXqCCDTHtw5l5kw4f4A4Ts2Fv3IncwfOlfLH9vT6vhNhrc_QArG9YbhgFyK/pub?w=1440&h=1080)

## National Land Cover Dataset

In [ ]:
styles = [
    dict(selector="th,td", props=[("font-size", "150%")])
]
legend.set_index("Code", inplace=True)
legend.style.applymap(highlight).set_table_styles(styles)

In [ ]:
len(legend)

In [ ]:
!du -sh /data/kotfic/nlcd_2011_landcover_2011_edition_2014_10_10.tif

In [ ]:
import fiona

fh = fiona.open("/data/kotfic/nynta-wgs84/nynta-wgs84.shp")

In [ ]:
for feature in fh:
    if feature['geometry']['type'] == "Polygon" and feature['properties']['BoroName'] == 'Manhattan':
        M.add_annotation('polygon', feature['geometry']['coordinates'][0], feature['properties'])

In [ ]:
p = M.layers.annotation.polygons[7]
p

In [ ]:
p.NTAName

In [ ]:
l, d = next(p.data)
d

In [ ]:
from collections import Counter

counts = zip(*np.unique(next(p.data)[1].data, return_counts=True))

print(p.NTAName)

data, index = zip(*[(num, legend.loc[c, 'Desc']) for c, num in counts if c != 0])
pd.Series(data, index=index, name="Count").to_frame()\
    .sort_values("Count", ascending=False)\
    .style.set_table_styles(styles)


In [ ]:
df = pd.DataFrame([(p.NTAName, n) for p in M.layers.annotation.polygons 
                     for n in next(p.data)[1].compressed()],
                    columns=["Neighborhood", "Code"])

In [ ]:
n_idx = df['Code'].isin([24])
d_idx = df['Code'].isin([21, 22, 23, 24])

high_dev_codes = df[n_idx].groupby('Neighborhood').sum()

all_codes  = df.groupby('Neighborhood').sum()

ddf = (high_dev_codes / all_codes).fillna(0.0).rename(columns={"Code": "High/All"})
ddf.sort_values("High/All", ascending=False).style.set_table_styles(styles)

### Don't forget to take a screen shot!

In [ ]:
M.layers.annotation.clear_annotations()
M.remove_layer(M.layers[0])

# Raster operations on the map

In this section we'll take a look at using the built in tile server to render raster data to the map. The tile server used is based on [KTile](https://github.com/OpenGeoscience/KTile) a fork of TileStache and is directly integrated into the Jupyter Notebook. The GeoJS map uses this tile server to render data efficiently to the map for visualization. 

In [ ]:
# Set the center of the map to the location the data
M.set_center(-120.32, 47.84, 7)

In [ ]:
from geonotebook.wrappers import RasterData

rd = RasterData('file:///data/kotfic/L57.Globe.month09.2010.hh09vv04.h6v1.doy247to273.NBAR.v3.0.tiff')
rd

### Adding a single band with JET colormap

In [ ]:
M.add_layer(rd[4], opacity=0.8)

### Something a little less agressive

In [ ]:
M.remove_layer(M.layers[0])

cmap = plt.get_cmap('winter', 10)
M.add_layer(rd[4], colormap=cmap, opacity=0.8)

### Something more appropriate for NDVI

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

M.remove_layer(M.layers[0])

# Divergent Blue to Beige to Green colormap
cmap =LinearSegmentedColormap.from_list(
  'ndvi', ['blue', 'beige', 'green'], 20)

# Add layer with custom colormap
M.add_layer(rd[4], colormap=cmap, opacity=0.8, min=-1.0, max=1.0)

# What can I do with this data?

We will address the use of annotations for analysis and data comparison in a separate notebook.  For now Let's focus on a small agricultural area north of I-90:

In [ ]:
M.layers.annotation.clear_annotations()

In [ ]:
M.set_center(-119.25618502500376, 47.349300631765104, 11)

In [ ]:
layer, data = next(M.layers.annotation.rectangles[0].data)
data

As a sanity check we can prove the data is the region we've selected by plotting the data with matplotlib's [imshow](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow) function:

*Note* The scale of the matplotlib image may seem slightly different than the rectangle you've selected on the map.  This is because the map is displaying in [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator) projection (EPSG:3857) while imshow is simply displaying the raw data, selected out of the geotiff (you can think of it as being in a 'row', 'column' projection).

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(16, 16))
ax.imshow(data, interpolation='none', cmap=cmap, clim=(-1.0, 1.0))

### NDVI Segmentation analysis

Once we have this data we can run arbitrary analyses on it.  In the next cell we use a sobel filter and a watershed transformation to generate a binary mask of the data.  We then use an implementation of marching cubes to vectorize the data,  effectively segmenting green areas (e.g. fields)  from surrounding areas.

This next cell requires both [scipy](https://www.scipy.org/) and [scikit-image](http://scikit-image.org/). Check your operating system documentation for how best to install these packages.

In [ ]:
# Adapted from the scikit-image segmentation tutorial
# See: http://scikit-image.org/docs/dev/user_guide/tutorial_segmentation.html
import numpy as np

from skimage import measure
from skimage.filters import sobel
from skimage.morphology import watershed
from scipy import ndimage as ndi


WATER_MIN = 0.2
WATER_MAX = 0.6

def print_segments(data, THRESHOLD = 20):

    fig, ax = plt.subplots(figsize=(10., 10.))
    edges = sobel(data)


    markers = np.zeros_like(data)
    markers[data > WATER_MIN] = 2
    markers[data > WATER_MAX] = 1


    mask = (watershed(edges, markers) - 1).astype(bool)
    seg = np.zeros_like(mask, dtype=int)
    seg[~mask] = 1

    # Fill holes
    seg = ndi.binary_fill_holes(seg)

    # Ignore entities smaller than THRESHOLD
    label_objects, _ = ndi.label(seg)
    sizes = np.bincount(label_objects.ravel())
    mask_sizes = sizes > THRESHOLD
    mask_sizes[0] = 0

    clean_segs = mask_sizes[label_objects]

    # Find contours of the segmented data
    contours = measure.find_contours(clean_segs, 0)
    ax.imshow(data, interpolation='none', cmap=cmap, clim=(-1.0, 1.0))

    ax.axis('tight')

    for n, contour in enumerate(contours):
        ax.plot(contour[:, 1], contour[:, 0], linewidth=4)
  
print_segments(data)

### Select a different region

In [ ]:
print_segments(next(M.layers.annotation.rectangles[1].data)[1].data)

In [ ]:
M.layers.annotation.clear_annotations()
M.remove_layer(M.layers[0])